In [16]:
#LIBRERÍAS

import os
import numpy as np
import cv2 
import time

In [17]:
#TRABAJANDO EN DIRECTORIO 
#os.chdir(r'C:\VIDEO')


In [35]:
#INICIALIZACIÓN DE VARIABLES
min_contour_width = 40  
min_contour_height = 40  
offset = 8 
line_height = 550  
matches = []
cars = 0

In [36]:
#CENTRO DEL OBJETO
def centro_id(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy
  

In [37]:
#LECTURA DEL VIDEO    
cap = cv2.VideoCapture('trafico.mp4')
ret, frame1 = cap.read()
 
#VideoCapture.read(). La función devuelve el cuadro de video. Si no se han tomado fotogramas, la imagen estará vacía.
if cap.isOpened():
    ret, frame1 = cap.read()
else:
    ret = False
   
ret, frame1 = cap.read()
ret, frame2 = cap.read()


In [38]:
cv2.imshow("Frame del video", frame1)
cv2.waitKey(0)

-1

In [39]:
while ret:
    
    #absdiff calcula la diferencia absoluta entre los elementos de una imagen o un escalar
    d = cv2.absdiff(frame1, frame2) #diferencia absoluta entre frames del video (matrices de imágenes)
    grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY) #traspaso a escala de grises
    
    #https://docs.opencv.org/3.4/d4/d13/tutorial_py_filtering.html
    blur = cv2.GaussianBlur(grey, (5, 5), 0) #filtro de suavizado
     
    #https://docs.opencv.org/3.4/d7/d4d/tutorial_py_thresholding.html
    #Si el valor de píxel es menor que el umbral, se establece en 0; de lo contrario, se establece en un valor máximo.
    ret, th = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY) #umbrales imagen SIEMPRE en escala de grises
    
   #https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html
   #Las transformaciones morfológicas son algunas operaciones simples basadas en la forma de la imagen.
   #Normalmente se realiza sobre imágenes binarias. Necesita dos entradas, una es nuestra imagen original, 
   #la segunda se llama elemento estructurante o núcleo que decide la naturaleza de la operación. 
   #Dos operadores morfológicos básicos son la erosión y la dilatación.
   
   #Dilatación: Aquí, un elemento de píxel es "1" si al menos un píxel en el kernel es "1". 
   #Por lo tanto, aumenta la región blanca en la imagen o aumenta el tamaño del objeto en primer plano. 
    dilated = cv2.dilate(th, np.ones((3, 3))) #dilatación de los bordes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2)) #kernel con forma de elipse
    
   #MorphologyEx es la dilatación seguida de la erosión. Es útil para cerrar pequeños agujeros dentro de los objetos
   #en primer plano o pequeños puntos negros en el objeto.
    closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
    
   #https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html
   #CHAIN_APPROX_SIMPLE Elimina todos los puntos redundantes y comprime el contorno, ahorrando así memoria.
    contours, h = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for(i, c) in enumerate(contours):
         #boundingRect es una función que se utiliza para crear un rectángulo aproximado junto con la imagen. 
       #El uso principal de esta función es resaltar el área de interés después de obtener la forma 
       #exterior de la imagen. Con las marcas adecuadas, los usuarios pueden resaltar fácilmente el 
       #aspecto deseado en una imagen. Por ejemplo, en el reconocimiento de rostros, 
       #después de reconocer el rostro, las imágenes se vuelven a dibujar utilizando un límite 
       #que cubre los rostros de las personas reconocidas en la foto
        (x, y, w, h) = cv2.boundingRect(c) #https://www.pythonpool.com/cv2-boundingrect/
       
        contour_valid = (w >= min_contour_width) and (h >= min_contour_height)
 
        if not contour_valid:
            continue
       
       #cv2.rectangle(image, start_point, end_point, color, thickness)
       #https://www.geeksforgeeks.org/python-opencv-cv2-rectangle-method/
        cv2.rectangle(frame1, (x-10, y-10), (x+w+10, y+h+10), (255, 0, 0), 2)
 
       #cv2.line(image, start_point, end_point, color, thickness)
       #https://www.geeksforgeeks.org/python-opencv-cv2-line-method/
        cv2.line(frame1, (0, line_height), (2000, line_height), (0, 255, 0), 2)
       
        centrolid = centro_id(x, y, w, h)
        matches.append(centrolid)
       
       #cv2.circle(image, center_coordinates, radius, color, thickness)
       #https://www.geeksforgeeks.org/python-opencv-cv2-circle-method/
        cv2.circle(frame1, centrolid, 5, (0, 255, 0), -1)
        cx, cy = centro_id(x, y, w, h)
       
        for (x, y) in matches:
            if y < (line_height+offset) and y > (line_height-offset):
                cars = cars+1
                matches.remove((x, y))
 
   #https://www.geeksforgeeks.org/python-opencv-cv2-puttext-method/ 
    cv2.putText(frame1, "Total de Autos Detectados: " + str(cars), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 170, 0), 2)
 
    cv2.imshow("Detectando vehiculos", frame1)
    if cv2.waitKey(1) == 55:
        break
   
    
    frame1 = frame2
    ret, frame2 = cap.read()
    
    time.sleep(0.050)


KeyboardInterrupt: 

In [40]:
cap.release()
cv2.destroyAllWindows()